In [1]:
import json

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

Saving spark_nlp_for_healthcare.json to spark_nlp_for_healthcare.json


In [2]:
%%capture
for k,v in license_keys.items(): 
    %set_env $k=$v

In [3]:
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_colab_setup.sh
!bash jsl_colab_setup.sh

! pip install spark-nlp-display

--2021-08-30 08:48:07--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1872 (1.8K) [text/plain]
Saving to: ‘jsl_colab_setup.sh’

jsl_colab_setup.sh  100%[===================>]   1.83K  --.-KB/s    in 0s      

2021-08-30 08:48:08 (30.4 MB/s) - ‘jsl_colab_setup.sh’ saved [1872/1872]

setup Colab for PySpark 3.1.1 and Spark NLP 3.1.3
Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRele

In [4]:
import json
import os
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl
import sparknlp

import pandas as pd

params = {"spark.driver.memory":"16G",
"spark.kryoserializer.buffer.max":"2000M",
"spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

Spark NLP Version : 3.1.1
Spark NLP_JSL Version : 3.1.3


In [5]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [6]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

sentenceDetector = SentenceDetectorDLModel.pretrained()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

tokenizer = Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("uncheckedtoken")\

spellModel = ContextSpellCheckerModel\
    .pretrained('spellcheck_clinical', 'en', 'clinical/models')\
    .setInputCols("uncheckedtoken")\
    .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

c2doc = Chunk2Doc()\
      .setInputCols("ner_chunk")\
      .setOutputCol("ner_chunk_doc") 

drugs_ner = MedicalNerModel.pretrained("ner_posology", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner_drug")\
    .setIncludeConfidence(False)

drugs_ner_converter = NerConverter() \
    .setInputCols(["sentence", "token", "ner_drug"]) \
    .setOutputCol("ner_chunk")\
    .setWhiteList(["DRUG"])

rxnorm_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_rxnorm","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("rxnorm_code")\
      .setDistanceFunction("EUCLIDEAN")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained("sbiobert_base_cased_mli",'en','clinical/models')\
      .setInputCols(["ner_chunk_doc"])\
      .setOutputCol("sbert_embeddings")
 
sbert_resolver_pipeline = Pipeline(
    stages = [
        documentAssembler,
        sentenceDetector,
        tokenizer,
        spellModel,
        word_embeddings,
        drugs_ner,
        drugs_ner_converter,
        c2doc,
        sbert_embedder,
        rxnorm_resolver])

data_ner = spark.createDataFrame([[""]]).toDF("text")

sbert_models = sbert_resolver_pipeline.fit(data_ner)

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
spellcheck_clinical download started this may take some time.
Approximate size to download 142.2 MB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[OK!]
sbiobertresolve_rxnorm download started this may take some time.
Approximate size to download 802.6 MB
[OK!]
sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]


In [12]:
with open('ocr_nlp_test3_di.txt', 'r') as file:
    sample = file.read()

In [8]:
light_rxnorm_model = LightPipeline(sbert_models)

In [13]:
light_data_rxnorm = light_rxnorm_model.fullAnnotate(sample)

In [14]:
a=[]
for rel in light_data_rxnorm[0]['rxnorm_code']:
  a.append((
      rel.metadata['target_text'],
      rel.result,
      rel.metadata['confidence'],
      rel.metadata['resolved_text'],
  ))
drugs = pd.DataFrame(a, columns=['chunk','rxnorm_code','confidence','rxnorm_code_name'])

In [15]:
drugs

,chunk,rxnorm_code,confidence,rxnorm_code_name
0,dc saline,9527,0.3397,saline
1,Vasopressors,57881,0.1223,vasocidin
2,Kristin,725028,0.0648,sudahist
3,Chantix,153806,0.0787,chimax
4,Zithromax,196474,0.3560,zithromax
5,albuterol-ipratropium,1154598,0.1674,albuterol / ipratropium inhalant product
6,NEB,203803,0.0761,nebcin
7,arformoterol,304962,0.4983,arformoterol
8,NEB,203803,0.0761,nebcin
9,Artificial Tears,9319,0.0607,rheomacrodex
